# Spotify Project 

#### Libraries

In [4]:
import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from time import sleep
from random import randint
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib
from IPython.display import IFrame

#### Importing the databases "Clustered song database": db_clustered with 14000 songs with 8 clusters, "top 100 songs":hot_songs from "https://www.billboard.com/charts/hot-100/"

In [5]:
db_clustered= pd.read_csv("clustered_songs_db.csv")
hot_songs= pd.read_csv("top_hundred_songs.csv")


#### Connecting to Spotify

In [6]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.client_id,
                                                           client_secret=config.client_secret))

#### Loading the model "kmeans": kmeans (Clustering algorithm), "scaling":"scaler" (Scaling algorithm) as Joblib file

In [7]:
kmeans = joblib.load('kmeans.gz')
scaler= joblib.load('scaler.gz')


#### User Functions 
###### "search track widget": To return the searched/user input song
###### "db_clustered_song_recomm(cluster)": To recommend a random song from the "clustered song database": db_clustered
###### "input_func(input)": To take user input and recommend a song from "top 100 songs": hot_songs or "clustered song database" : db_clustered

In [26]:
def search_track_widget(track):
    track= track.lower()
    track= sp.search(q=track,limit=1,type="track",market="GB")
    uri= track["tracks"]["items"][0]["uri"]
    uri_lstrip= uri.lstrip("spotify:track:")
    return IFrame(src=f"https://open.spotify.com/embed/track/{uri_lstrip}",
                    width="320",
                    height="80",
                    frameborder="0",
                    allowtransparency="true",
                    allow="encrypted-media",
                )
    

In [9]:
def db_clustered_song_recomm(cluster):
    result= (db_clustered[db_clustered["cluster"]==cluster]).sample()
    return result

In [10]:
def input_func(input):
    input= input.lower()
    if input in list(hot_songs["title"]):
        random=(hot_songs.sample())
        print("The song is in the hot list, try number",random["title"])
        track= sp.search(q=random["title"],limit=1,type="track",market="GB")
        uri= track["tracks"]["items"][0]["uri"]
        uri_lstrip= uri.lstrip("spotify:track:")
        return IFrame(src=f"https://open.spotify.com/embed/track/{uri_lstrip}",
                    width="320",
                    height="80",
                    frameborder="0",
                    allowtransparency="true",
                    allow="encrypted-media",
                )
       
    else:
        track= sp.search(q=input,limit=1,type="track",market="GB")
        audio_features= sp.audio_features(track["tracks"]["items"][0]["uri"])
        a= pd.DataFrame(audio_features)
        a= a._get_numeric_data()
        a= a.values.tolist()
        a_scaled= scaler.transform(a)
        cluster= kmeans.predict(a_scaled)
        rec_song= db_clustered_song_recomm(cluster[0])
        track= sp.search(q=rec_song["songs"],limit=1,type="track",market="GB")
        uri= track["tracks"]["items"][0]["uri"]
        uri_lstrip= uri.lstrip("spotify:track:")
        return IFrame(src=f"https://open.spotify.com/embed/track/{uri_lstrip}",
                    width="320",
                    height="80",
                    frameborder="0",
                    allowtransparency="true",
                    allow="encrypted-media",
                )
     
    

#### Running the "search track widget":search tracks, "User input function": input_func(input) for the user request song

In [40]:
user_input= input("Type a song you like!")
searched_track= search_track_widget(user_input)
searched_track

In [41]:
ip= input_func(user_input)
ip

E:\Anaconda\envs\DA_Envi\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\Anaconda\envs\DA_Envi\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
